In [ ]:
import numpy as np
import pandas as pd
from debugpy.launcher import channel
from scipy.signal import firwin, lfilter
from scipy import ndimage
import matplotlib.pyplot as plt
import seaborn as sns
import os
#os.chdir('/Users/admin/PycharmProjects/ET_eksploracja/hyperscanning-signal-analysis')
%matplotlib inline

In [ ]:
import sys
sys.path.append('../src')
import dataloader
from data_structures import MultiModalDataPd

In [ ]:
!pwd

In [ ]:
mmd = MultiModalDataPd("W003")
mmd.add_data(eeg_path = "../data/W003/eeg/")

In [ ]:
mmd.data.head()

In [ ]:
mmd.data.events.unique()

In [ ]:
mmd.data.info()

## Test plot of diode signal colored by events

In [ ]:
# Create a figure
fig, ax = plt.subplots(figsize=(12, 6))

# Get unique events for color mapping
events = mmd.data['events'].unique()
colors = plt.cm.tab10(np.linspace(0, 1, len(events)))
event_colors = dict(zip(events, colors))

# Plot DIODE values colored by events
for event in events:
    event_data = mmd.data[mmd.data['events'] == event]
    ax.plot(event_data['time'], event_data['DIODE'], label=event, color=event_colors[event])

ax.set_xlabel('Time')
ax.set_ylabel('DIODE')
ax.set_title('DIODE Signal over Time (colored by events)')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
mmd.add_data(et_path = "../data/et_data/")
#mmd.add_data(ecg_path = "../data/W003/ecg/")

In [ ]:
mmd.data.head()

In [ ]:
mmd.data.loc[mmd.data['ET_event'] == 'm2','ET_cg_x']

In [ ]:
mmd.data.columns

In [ ]:
mmd.data['events'].unique()

In [ ]:
mmd.data['ET_event'].unique()

In [ ]:
# Create a figure
fig, ax = plt.subplots(figsize=(12, 6))

# Get unique events for color mapping
events = mmd.data['events'].unique()
colors = plt.cm.tab10(np.linspace(0, 1, len(events)))
event_colors = dict(zip(events, colors))

# Plot DIODE values colored by events
for event in events:
    event_data = mmd.data[mmd.data['events'] == event]
    ax.plot(event_data['time'], event_data['DIODE'], label=event, color=event_colors[event])


# add plots of DIODE vs time_idx colored based on ET_events
et_events = mmd.data['ET_event'].unique()
et_colors = plt.cm.tab10(np.linspace(0, 1, len(et_events)))
et_event_colors = dict(zip(et_events, et_colors))

for et_event in et_events:
    et_event_data = mmd.data[mmd.data['ET_event'] == et_event]
    ax.plot(et_event_data['time'], et_event_data['DIODE']+0.1, label=f'ET: {et_event}', color=et_event_colors[et_event], linestyle='-', alpha=0.7)

ax.set_xlabel('Time')
ax.set_ylabel('DIODE')
ax.set_title('DIODE Signal over Time (colored by events)')
ax.legend()
plt.tight_layout()
#plt.xlim(120, 180)
plt.show()

## Exaple of selecting EEG Fz, ECG and ET y and blinks  data form 'Pepa' event

In [ ]:
%matplotlib inline
data_selected = mmd.data[(mmd.data.events == 'Incredibles')]
fig, ax = plt.subplots(figsize=(12, 6),sharex=True, sharey=False,nrows=4)
ax[0].plot(data_selected['time'], data_selected['ET_cg_y'], linestyle='-', label='ET_cg_y')
ax[0].plot(data_selected['time'], data_selected['ET_ch_y'], linestyle='-', label='ET_ch_y')
ax[0].legend()

ax[1].plot(data_selected['time'], data_selected['ET_cg_blinks'], linestyle='-', label='ET_cg_blinks')
ax[1].plot(data_selected['time'], data_selected['ET_ch_blinks'], linestyle='-', label='ET_ch_blinks')
ax[1].legend()

ax[2].plot(data_selected['time'], data_selected['EEG_cg_Fp1'], linestyle='-', label='EEG_cg_Fp1')
ax[2].plot(data_selected['time'], data_selected['EEG_ch_Fp1'], linestyle='-', label='EEG_ch_Fp1')
ax[2].legend()

ax[3].plot(data_selected['time'], data_selected['DIODE'], linestyle='-', label='DIODE')
ax[3].legend()
#plt.xlim(0,20)
plt.xlabel('Time')
plt.show()